<a href="https://colab.research.google.com/github/DanielHevdeli/hafifot-tiug/blob/main/LLMs_as_annotators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!git clone https://github.com/DanielHevdeli/hafifot-tiug.git

fatal: destination path 'hafifot-tiug' already exists and is not an empty directory.


In [9]:
pip install dspy

In [1]:
import pandas as pd
import os
from typing import Literal

In [2]:
posts_df = pd.read_csv('./hafifot-tiug/data/split_data/present.csv')

In [3]:
posts_df.head()

,question_id,length,date,text
0,114678,903,2016-04-10 20:05:00,"שלום , אני מאוד מקווה שתוכלו לעזור לי אני לא י..."
1,116349,888,2016-04-24 19:56:00,היי כולם \nיש לי בעיה הקשורה לתספורת שלי. \nאמ...
2,108590,1183,2016-02-16 14:23:00,הגעתי לאתר הזה במקרה אחרי שקראתי טיפה מפוסטים ...
3,100709,2689,2015-12-15 11:33:00,"בערך מגיל 17 אני סובלת מחרדה, כאשר חלה החרפה ק..."
4,120766,1112,2016-05-27 22:26:00,"אני תוהה כבר כמה שנים מה ללמוד בעתיד, איזה מקצ..."


In [4]:
posts_df.iloc[3]['text']

'בערך מגיל 17 אני סובלת מחרדה, כאשר חלה החרפה קיצונית בגיל 18 עקב לימודים ומצב חברתי גרוע והשפלות יומיומיות  . בהתחלה לא הבנתי בכלל שמדובר בחרדה כי הסימנים היו פיזיולוגים והחששות תמיד היו. התחלתי "להתעלף"  , הרגשתי שאני לא מצליחה לנשום וגם כשאני כן מצליחה אז האוויר לא נכנס לי לריאות , היו לי סחרחורות שהייתי בטוחה שאני עומדת למות ולהתמוטט. נשלחתי למיון וכאשר לא מצאו לי כלום הניחו שזה חרדה.ככה העברתי את השנה האחרונה שלי בבית ספר, כיתה יב. כלומר, במיטה ובדיכאון ללא חברים ולומדת לבגרויות. \nכמה חודשים אחר כך עליתי על מדים והאמת שמצבי השתפר פלאים. אומנם החרדה לא נעלמה , ותמיד היו ניצוצות של חרדה אבל הצלחתי לעבור אותן. \nכהשתחררתי היה לי מין תהום ענקית , הרגשתי בחופש ורציתי כל היום לישון,מה שבצבא לא התאפשר לי וגם כן לא לפני . יצאתי מדי פעם עם חברות אבל גם זה אחכ הפסיק. אחרי כמה חודשים כאלה התחלתי את הפסיכומטרי ולצערי בגלל תחושות של חרדה והתמוטטות בשיעור דחיתי את המועד. אחרי הדחייה נכנסתי לדכאון וחרדה קשים , לא רציתי לצאת מהמיטה.אם פעם התקפים באו והלכו אז המצב שלי בחודשיים האחרונים היו 24/7. 

In [5]:
posts_df.iloc[2]['text']

'הגעתי לאתר הזה במקרה אחרי שקראתי טיפה מפוסטים של אנונימים פה—אני מקווה שגם אוכל למצוא פה מענה או כיוון לפתרון לבעיה שלי.  \nאני לא יודעת כל כך מאיפה להתחיל או מאיפה התחיל הסיפור אבל אני סובלת בערך כל חיי מחרדה ודיכאון. \nהייתי מודעת לעניין אבל מעולם לא חוויתי מצב קיצוני של חרדה משתקת לחלוטין. היו לי התקפים מזעזעים שבגללם הלכתי למיון.  \nאבל בקיצור, בתקופה האחרונה של כמעט שנה אני חוויתי אגרופוביה רצינית עם התקפי דכאון ומחשבות מזעזעות. ברמה של סיעוד. לא התקלחתי, כל היום הייתי ישנה ומסוגרת מתחת לשמיכה. \nהפסיכאטר נתן לי ציפרלקס .  \nאני מרגישה שיפור עם הדיכאון יחסית, ועם ההתקפי חרדה אבל עד היום האגרובופיה לא נעלמה, קשה לי לתפקד. אני רוצה לצאת החוצה,להכיר אנשים, לעבוד, לקבל רישיון, ללמוד. לא להיות למה שהפכתי. \nמיותר לציין,שניתקתי קשר עם כל סבוביי כך שאין לי חיי חברה.  \nאת כל הכסף שאין לי אני מוציאה על מוניות בדרך לפסיכאטר ובחזרה מימנו כי קשה לי לעלות על אוטובוסים. \nהפסקתי עם השיעורי נהיגה בגלל ההתקפים בנהיגה .  \nבקיצור,אם המצב ימשיך אני לא יודעת איך אוכל לשרוד ככה,גם מבחינה כספית וגם 

Let's try to classify each post to either suicidal or non-suicidal. It may help the publishers of the website to offer first-help to writers of post categorized as suicidal even before other people answer them.

# Annotate Posts

In [6]:
import dspy

In [32]:
# MODEL_NAME = "huggingface/Norod78/hebrew-gpt_neo-tiny"
lm = dspy.LM(model="openai/gpt-3.5-turbo", )

In [25]:
dspy.configure(lm=lm)

In [33]:
class Classify(dspy.Signature):
    """Classify label of a given post."""

    post: str = dspy.InputField()
    label: Literal["suicidal", "non-suicidal"] = dspy.OutputField()

In [34]:
classify = dspy.Predict(Classify)

In [35]:
classify(post="היי אני חייב עזרה דחופה בהרצת ההתקנה של הספריה")

AuthenticationError: litellm.AuthenticationError: AuthenticationError: OpenAIException - Incorrect API key provided: "sk-proj**********************************************************************************************************************************************************3EA". You can find your API key at https://platform.openai.com/account/api-keys.

In [ ]:
posts_labels = []
i = 0
for index, row in posts_df.iterrows():
    question_id = row['question_id']
    text = row['text']

    prediction = classify(post=text)
    label = prediction.label

    posts_labels.append({'question_id': question_id, 'label': label})

print(f"Classification of {len(posts_labels)} posts completed.")

In [ ]:
classified_df = pd.DataFrame(posts_labels)
# save to hafifot-tiug/data/labels/present_labels_{model_name}.csv


In [ ]:
labels_df.head(10)

In [ ]:
# or in s single cell:
'''
import pandas as pd
import os
from typing import Literal
import dspy

CLASSES = Literal['suicidal', 'non-suicidal']
posts_df = pd.read_csv('./hafifot-tiug/data/split_data/present.csv')
annotator_output_dir = './hafifot-tiug/data/annotators_labels/present'
os.makedirs(annotator_output_dir, exist_ok=True)

dspy.configure(lm=dspy.LM(model="openai/gpt-3.5-turbo"))

class Classify(dspy.Signature):
    """Classify label of a given post."""

    post: str = dspy.InputField()
    label: Literal["suicidal", "non-suicidal"] = dspy.OutputField()

classify = dspy.Predict(Classify)

def classify_post(post: str) -> CLASSES:
    result = classify(post=post)
    return result.label

def save_annotator_labels(annotator_name: str, df: pd.DataFrame):
    output_file = os.path.join(annotator_output_dir, f'{annotator_name}_labels.csv')
    df.to_csv(output_file, index=False)
    print(f"Saved labels for {annotator_name} to {output_file}")

def annotate_with_llm(annotator_name: str, posts_df: pd.DataFrame):
    labels = []
    for _, row in posts_df.iterrows():
        post = row['text']
        label = classify_post(post)
        labels.append({'question_id': row['question_id'], 'label': label})
    labels_df = pd.DataFrame(labels)
    save_annotator_labels(annotator_name, labels_df)

if __name__ == "__main__":
    annotate_with_llm('gpt-4', posts_df)
'''